### 직방의 원룸 매물정보 수집
- 절차
    - 동이름으로 위도, 경도 수집
    - 위도, 경도 > geohash(영역) 변환
    - geohash(영역) > 매물 아이디
    - 매물 아이디 > 매물 정보 수집

In [5]:
import requests
import pandas as pd
import geohash2

In [4]:
!pip install geohash2 # geohash2 설치

  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15544 sha256=39ccd69c7ea4bc309b290018969acf9b1f51d0ef84696c735a0218638056f6cb
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\2d\0c\66\dcd768c9e7c26fc81ec59bfe24af9d780fc9dbdf7f90b0cf00
Successfully built geohash2


In [1]:
# 1. 동이름 > 위도, 경도
# zigbang 홈페이지 > 원룸 > 개발자 도구 > Network > Fetch/XHR

In [8]:
addr = "연제구 연산동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(35.182559967041016, 129.08554077148438)

In [2]:
# 2. 위도, 경도 > geohash

In [10]:
# precsision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wy7b1'

In [ ]:
# 3. geohash > 매물 아이디

In [11]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
# \로 문자열을 이어줄 때 ' ' 공백이 있으면 에러 발생하니 주의
response = requests.get(url)
response

<Response [200]>

In [15]:
items = response.json()["items"]
# 실제로 가져올 수 있는 개수가 1000개가 넘는 경우가 있다
# 데이터가 1000이 넘어가는 경우 parameter를 넘길 때 나누어 넘겨야 한다
len(items), items[:2]

(699,
 [{'lat': 35.15730458039277, 'lng': 129.11074406994152, 'item_id': 32650687},
  {'lat': 35.15783308484765, 'lng': 129.11026280206823, 'item_id': 32727230}])

In [16]:
ids = [item["item_id"]for item in items]
ids[:5]

[32650687, 32727230, 32838086, 32523449, 32227873]

In [ ]:
# 4. 매물 아이디 > 매물 정보

In [17]:
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain" : "zigbang",
    "withCoalition" : "true",
    "item_ids" : ids
    #"item_ids" : ids[:900]
}
response = requests.post(url, params)
response

<Response [200]>

In [23]:
items = response.json()["items"]
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
df = pd.DataFrame(items)[columns]
df.head(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32650687,월세,500,40,18.00,부산시 수영구 광안동,14
1,32727230,월세,300,45,26.45,부산시 수영구 광안동,10
2,32838086,월세,7000,20,29.75,부산시 수영구 광안동,15
3,32523449,월세,100,16,26.97,부산시 부산진구 전포동,10
4,32227873,월세,100,10,33.06,부산시 부산진구 전포동,10
...,...,...,...,...,...,...,...
694,32829925,월세,100,15,14.61,부산시 동래구 안락동,10
695,32783323,월세,500,43,26.45,부산시 동래구 안락동,10
696,32811035,월세,2000,50,42.98,부산시 동래구 안락동,10
697,32775937,전세,11000,0,21.56,부산시 동래구 사직동,7


In [ ]:
# max row, max column 설정
# pd.options.display.max_columns = 50
# 위와 같이 설정하면 열 혹은 행이 너무 많아 ... 으로 생략되는 경우
# ... 이 아닌 원래의 데이터프레임을 확인할 수 있다

In [24]:
# 함수 만들기
def oneroom(addr) :
    # 위도 경도 알아오기
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    # 위도, 경도 값을 geohash로 변환
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # geohash를 통해 매물id를 가져온다
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    # \로 문자열을 이어줄 때 ' ' 공백이 있으면 에러 발생하니 주의
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"]for item in items]
    
    # 매물id를 통해서 매물에 대한 정보를 가져온다
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain" : "zigbang",
        "withCoalition" : "true",
        "item_ids" : ids
        #"item_ids" : ids[:900]
    }
    response = requests.post(url, params)
    items = response.json()["items"]
    # 필요한 정보들만 따로 선별해서 가져올 수 있게 한다
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    df = pd.DataFrame(items)[columns]
    return df

In [25]:
# 마포구 합정동을 통해 함수 확인
addr = "마포구 합정동"
df = oneroom(addr)
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
742,32862490,전세,43000,0,31.74,서울시 마포구 중동,5
743,32866975,월세,3000,90,49.59,서울시 마포구 중동,2


In [27]:
# addr과 찾아온 데이터들의 address1이 같은 것만 필터링
df_filtered = df[df["address1"].str.contains(addr)].reset_index(drop=True)
df_filtered

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32662713,전세,17700,0,39.67,서울시 마포구 합정동,0
1,32668759,전세,17700,0,39.67,서울시 마포구 합정동,0
2,32726690,전세,19900,0,36.28,서울시 마포구 합정동,0
3,32542700,월세,1500,95,49.59,서울시 마포구 합정동,7
4,32513282,전세,5000,0,23.14,서울시 마포구 합정동,5
...,...,...,...,...,...,...,...
95,32794268,월세,1000,90,59.50,서울시 마포구 합정동,2
96,32794303,월세,3000,80,59.50,서울시 마포구 합정동,2
97,32804404,월세,2000,85,61.24,서울시 마포구 합정동,2
98,32812101,월세,3000,80,61.24,서울시 마포구 합정동,2


In [28]:
# 모듈 파일 만들기 : py

In [32]:
%ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 7C6C-FC35

 C:\Users\User\Desktop\교안\2022_08_04\Code 디렉터리

2022-08-04  오후 01:33    <DIR>          .
2022-08-04  오후 01:33    <DIR>          ..
2022-08-04  오전 11:08    <DIR>          .ipynb_checkpoints
2022-08-04  오전 11:07            52,316 01_naver_api_2.ipynb
2022-08-04  오후 01:31            23,893 02_zigbang.ipynb
2022-08-04  오후 01:33             1,456 zigbang.py
               3개 파일              77,665 바이트
               3개 디렉터리  123,750,555,648 바이트 남음


In [31]:
%%writefile zigbang.py
import requests
import pandas as pd
import geohash2

def oneroom(addr) :
    # 위도 경도 알아오기
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    # 위도, 경도 값을 geohash로 변환
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # geohash를 통해 매물id를 가져온다
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    # \로 문자열을 이어줄 때 ' ' 공백이 있으면 에러 발생하니 주의
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"]for item in items]
    
    # 매물id를 통해서 매물에 대한 정보를 가져온다
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain" : "zigbang",
        "withCoalition" : "true",
        "item_ids" : ids
        #"item_ids" : ids[:900]
    }
    response = requests.post(url, params)
    items = response.json()["items"]
    # 필요한 정보들만 따로 선별해서 가져올 수 있게 한다
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    df = pd.DataFrame(items)[columns]
    return df

Writing zigbang.py


In [33]:
import zigbang as zb

In [34]:
df = zb.oneroom("금정구 남산동")
df.head()

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,31938155,월세,500,30,27.39,부산시 금정구 구서동,10
1,32875488,월세,100,14,20.23,부산시 금정구 구서동,7
2,32837621,월세,500,40,29.13,부산시 금정구 구서동,10
3,32680930,월세,300,30,19.83,부산시 금정구 구서동,7
4,32243059,월세,4000,40,40.86,부산시 금정구 구서동,10


In [36]:
# 사용하는 변수 목록을 확인할 수 있다
%whos
# 위의 변수 목록을 초기화한다
%reset
%whos

Variable      Type         Data/Info
------------------------------------
addr          str          마포구 합정동
columns       list         n=7
data          dict         n=13
df            DataFrame          item_id sales_type <...>n\n[146 rows x 7 columns]
df_filtered   DataFrame         item_id sales_type  <...>n\n[100 rows x 7 columns]
geohash       str          wy7b1
geohash2      module       <module 'geohash2' from '<...>\\geohash2\\__init__.py'>
ids           list         n=699
items         list         n=699
lat           float        35.182559967041016
lng           float        129.08554077148438
oneroom       function     <function oneroom at 0x000001B35DDD3EE0>
params        dict         n=3
pd            module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests      module       <module 'requests' from '<...>\\requests\\__init__.py'>
response      Response     <Response [200]>
url           str          https://apis.zigbang.com/v2/items/list
zb           